In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
for dirname, _, filenames in os.walk('../input/covid19-image-dataset/Covid19-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
batch_size = 20
num_workers = 0

data_dir = '../input/covid19-image-dataset/Covid19-dataset'
train_dir = os.path.join(data_dir, 'train')

test_dir = os.path.join(data_dir, 'test')
data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225]) ]),
                   'val': transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])]),
                   'test': transforms.Compose([transforms.Resize(size=(224,224)),
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])])
                  }

train_data = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=False)
loaders_scratch = {
    'train': train_loader,
    'test': test_loader
}

../input/covid19-image-dataset/Covid19-dataset/test/Normal/0121.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0109.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0117.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0102.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0107.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0106.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0111.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0114.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0118.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0122.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0119.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0103.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0112.jpeg
../input/covid19-image-dataset/Covid19-dataset/test/Normal/0105.jpeg
../input/covid19-image-dataset/Cov

In [3]:
import torch
import torchvision.models as models
use_cuda = torch.cuda.is_available()

In [4]:
## TODO: Specify data loaders
loaders_transfer=loaders_scratch.copy()

In [37]:
import torchvision.models as models
import torch.nn as nn

## TODO: Specify model architecture 

# Select a pre-trained VGG19 CNN
model_transfer=models.resnet101(pretrained=True)



KeyboardInterrupt: 

In [27]:
model_transfer

if use_cuda:
    model_transfer = model_transfer.cuda()

In [29]:
import torch.optim as optim
import torch.nn as nn
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.parameters(), lr=0.005)

In [30]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            
            #clearing the gradients
            optimizer.zero_grad()
            
            #predicted output
            output=model(data)
            
            
            loss = criterion(output, target)
            # backward propogation
            loss.backward()
            
            optimizer.step()
            # update training loss
            
            
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            
            if batch_idx % 100 == 0:
                print('Epoch %d, Batch %d loss: %.6f' %
                  (epoch, batch_idx + 1, train_loss))
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['test']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss

            # forward pass
            output = model(data)
            # loss
            loss = criterion(output, target)
            # update validation loss
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
          
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
      
    if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).    Saving model...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        
    # return trained model
    return model

In [31]:
model_transfer = train(5, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

Epoch 1, Batch 1 loss: 9.217237
Epoch: 1 	Training Loss: 4.400350 	Validation Loss: 2.709507
Epoch 2, Batch 1 loss: 0.976606
Epoch: 2 	Training Loss: 0.625608 	Validation Loss: 1.261217
Epoch 3, Batch 1 loss: 0.404602
Epoch: 3 	Training Loss: 0.400272 	Validation Loss: 0.562432
Epoch 4, Batch 1 loss: 0.258950
Epoch: 4 	Training Loss: 0.387061 	Validation Loss: 1.182591
Epoch 5, Batch 1 loss: 0.589598
Epoch: 5 	Training Loss: 0.306122 	Validation Loss: 0.441539
Validation loss decreased (inf --> 0.441539).    Saving model...


<All keys matched successfully>

In [32]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [33]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

Test Loss: 0.441539


Test Accuracy: 87% (58/66)
